<h1>PRÁCTICA 1 - FUNDAMENTOS DE APRENDIZAJE AUTOMÁTICO</h1>
<h3>Realizada la práctica por:<br/>
    <ol>
    -Pablo Díez del Pozo<br/>
    -Alejandro Alcalá Álvarez
    </ol>
 </h3>
<h3>Grupo: 1461</h3>
<h3>Pareja: 01</h3>

In [1]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

<h3>Importaciones necesarias para la ejecucion del código</h3>
<p>Podemos observar todos los import necesarios que tenemos que realizar para que la ejecución de nuestro codigo funcione a la perfección, a continuación, explicaremos cada uno de los imports y para que son necesarios:</p>
<ol>
    <li>Random: se utiliza para hacer las secuencias de índices aleatorios para las particiones de entrenamiento y de clasificación.
    <li>Math: se utiliza para hacer la distribución normal para los atributos que sean continuos y asi poder calcular su probabilidad.
    <li>Numpy: Es la libreria mas utilizada en esta práctica, debido a que almacenamos los datos en una matriz numpy y guardamos las probabilidades posterioris de los atributos en un array de matrices de numpy.
    <li>ABC: se utiliza para haces clases y métodos abstractos.
    <li>Datos: se utiliza para importar toda la funcionalidad de nuestro modulo Datos.
    <li>Collections: se utiliza para contabilizar las probabilidades condicionadas y para ver cuantas clases hay en el fichero
    <li> SortedDict: se utiliza para ordenar el diccionario que creamos con las probabilidades a priori de cada clase
    <li>Sklearn: se utiliza para hacer el tercer apartado de esta práctica, donde nos da una implementación del algoritmo de Naive-Bayes
    <li>Pyplot: se utiliza en el último apartado de la práctica, donde nos da una implementación para pintar la curva ROC.
     

In [18]:
import random
import math
import numpy as np
from abc import ABCMeta, abstractmethod
from Datos import Datos
from collections import Counter
from sortedcontainers import SortedDict
from sklearn.metrics import confusion_matrix, auc
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

<h3>Obtener los datos de los Distintos Dataset</h3>
<p>Aqui vamos a poder observar como vamos a codificar los datos que nos dan en un fichero a una matriz Numpy para poder tratar los datos para poder entrenarlos y clasificarlos con Naive-Bayes</p>
<p>Vamos a ver como llamando a la clase Datos y que en su constructor le ponemos la ruta del fichero se crea la matriz numpy de los datos, pero a demás de esa matriz también guardamos información necesaria para poder entrenarlos y clasificarlos correctamente. Por ejemplo, guardamos si los atributos son continuos o discretos.</p>
<p>A continuación, vamos a mostrar una ejecución para cada uno de los conjuntos de datos que nos dan para hacer Naive-Bayes. En la celda de abajo vereis la ejecución.</p>

In [4]:
dataset = Datos('../Datasets/lenses.data')
dataset2 = Datos('../Datasets/tic-tac-toe.data')
dataset3 = Datos('../Datasets/german.data')
print("==============MATRIZ NUMPY DEL CONJUNTO DE DATOS LENSES=====================")
print(dataset.datos)
print("============================================================================")
print("==============MATRIZ NUMPY DEL CONJUNTO DE DATOS TIC-TAC-TOE================")
print(dataset2.datos)
print("============================================================================")
print("==============MATRIZ NUMPY DEL CONJUNTO DE DATOS GERMAN=====================")
print(dataset3.datos)
print("============================================================================")

['Age', 'Spectacle', 'Astigmatic', 'Tear', 'Class']
[{'1': 0, '2': 1, '3': 2}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1}, {'1': 0, '2': 1, '3': 2}]
['TLeftSq', 'TMidSq', 'TRightSq', 'MLeftSq', 'MMidSq', 'MRightSq', 'BLeftSq', 'BMidSq', 'BRightSq', 'Class']
[{'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'b': 0, 'o': 1, 'x': 2}, {'negative': 0, 'positive': 1}]
['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'Class']
[{'A11': 0, 'A12': 1, 'A13': 2, 'A14': 3}, {}, {'A30': 0, 'A31': 1, 'A32': 2, 'A33': 3, 'A34': 4}, {'A40': 0, 'A41': 1, 'A410': 2, 'A42': 3, 'A43': 4, 'A44': 5, 'A45': 6, 'A46': 7, 'A48': 8, 'A49': 9}, {}, {'A61': 0, 'A62': 1, 'A63': 2, 'A64': 3, 'A65': 4}, {'A71': 0, 'A72': 1, 'A73': 2, 'A74': 3, 'A75': 4}, {}, {'A91': 0,

<h3>Apartado 1: Estrategia de Particionado</h3>
<p>En este apartado vamos a probar las dos estrategias de particionado de los datos que hemos tenido que implementar en esta práctica, las cuales son:</p>
    <ol>
        <p>- Validación Simple.</p>
        <p>- Validación Cruzada.</p>
    </ol>
<p>Nuestra estrategia de <strong>validación simple</strong> consiste en meterle un porcentaje por el cual queremos dividir el conjunto de datos en dos subconjuntos de datos, donde uno lo vamos a utilizar para entrenar y el otro lo vamos a utilizar para hacer la predicción con nuestro clasificador. En la celda de abajo mostraremos el código necesario para poder realizar correctamente la validacion simple.</p>
<p>Como podemos observar en el código de abajo de validación simple, lo que hacemos es que ponemos una semilla a random y decimos que el numero de particiones va a ser uno. A continuación, haremos un permutacion de numeros aleatorios entre el 0  y el número de datos que hay en el fichero. Por ultimo, lo que hacemos es que le creamos la partición que va a tener en su interior los dos subconjuntos de Train y Test. En esa permutación lo multiplicamos por el porcentaje que le hemos dado nosotros para crear los dos suboconjuntos.</p>
<p>Debajo de esta celda vamos a comprobar en como funciona  la validación simple con diferentes porcentajes para obtener el subconjunto de datos de Train y Test, en los diferentes conjuntos de datos </p>

In [5]:
class Particion():

  # Esta clase mantiene la lista de �ndices de Train y Test para cada partici�n del conjunto de particiones
  def __init__(self, train=[], test=[]):
    self.indicesTrain = train
    self.indicesTest = test

  def __str__(self):
    return "Train: {}\nTest:  {}".format(str(self.indicesTrain), str(self.indicesTest))

class EstrategiaParticionado:
  # Clase abstracta
  __metaclass__ = ABCMeta

  # Lista de las particiones
  def __init__(self, nombre=""):
    self.nombreEstrategia = nombre
    self.numeroParticiones = 0
    self.particiones = []

  # Atributos: deben rellenarse adecuadamente para cada estrategia concreta: nombreEstrategia, numeroParticiones, listaParticiones. Se pasan en el constructor

  @abstractmethod
  # TODO: esta funcion deben ser implementadas en cada estrategia concreta
  def creaParticiones(self, datos, seed=None):
    pass

class ValidacionSimple(EstrategiaParticionado):

  def __init__(self, porcentaje):
    self.porcentaje = porcentaje
    super().__init__("Validacion simple")

  # Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
  # Devuelve una lista de particiones (clase Particion)
  # TODO: implementar
  def creaParticiones(self, datos, seed=None):

    np.random.seed(seed)
    self.numeroParticiones = 1

    # Generamos una lista con todos los números de datos aleatorios
    indicesAleatorios = np.random.permutation(int(datos.numDatos))

    # Creamos la particion, en funcion del porcentaje especificado
    self.particiones = [Particion(indicesAleatorios[:int(datos.numDatos * self.porcentaje)],
                                  indicesAleatorios[int(datos.numDatos * self.porcentaje):])]

    return self.particiones

In [6]:
print("==============VALIDACIÓN SIMPLE CON LENSES DATA 70%======================")
estrategia = ValidacionSimple(0.7)
estrategia.creaParticiones(dataset)
print(estrategia.particiones[0])
print("=========================================================================")
print("==============VALIDACIÓN SIMPLE CON TIC-TAC-TOE DATA 80%=================")
estrategia2 = ValidacionSimple(0.8)
estrategia2.creaParticiones(dataset2)
print(estrategia2.particiones[0])
print("=========================================================================")
print("==============VALIDACIÓN SIMPLE CON GERMAN DATA 75%======================")
estrategia3 = ValidacionSimple(0.75)
estrategia3.creaParticiones(dataset3)
print(estrategia3.particiones[0])
print("=========================================================================")


==============VALIDACIÓN SIMPLE CON LENSES DATA 70%======================
Train: [ 3  0  9 23 21  4 10  6 11 13  8 17 14  1 22 20]
Test:  [16  7 12  2 18  5 15 19]
==============VALIDACIÓN SIMPLE CON TIC-TAC-TOE DATA 80%=================
Train: [929  18 772 957 107 393 323 867 763 443 503  32 804 603 109 828 750 147
 442  33 381  13 229 457 552 557  69 303 454 398 436 166 285   2 295 605
 344 920 169 210 659 440 576 139 712 545 445 905 741 821 827 207 111 129
 495 433 376 273 815 126 224 608 908 667 756  52 296 548 331 184 149 396
  60 641 842 466 415 910 671  61 843 743 197 242 182 661 554 424 322 688
 761 274 535 214  54 318 131 232  41 609 725  24 730  46 681 762 279 482
   0 651 140 868 211 379  82 838 145 215 700 218 135 357 736 653 846 595
 600 585 807 572 735 921 470 231  77 309  26 581 745 356 464  89 348 112
 226 350   5 277 187 638 458 148  45 955 267 592 930 160 555 740 372 243
 161  72 387 254 333 724 791 720 690 395 562 392 947 680 676 626 246 198
 787 263 954  92 672 748 

<p>Nuestra estrategia de <strong>validación cruzada</strong> consiste en dividir nuestro conjunto de datos en particiones de Train y Test como en validación simple, pero este proceso lo vamos a hacer K veces, para que todos los datos esten presenten en los dos subconjunto de datos para poder obtener una mejora a la hora de entrenar y clasificar.A continuación, mostraremos nuestra implementación de la validación cruzada; donde vamos a hacer K veces las divisiones del conunto de datos y si por algún motivo nuestra división de todos los subconjuntos no es perfecta balancearemos los datos sobrantes para poder entrenarlos y clasificarlos.</p>

In [7]:
class ValidacionCruzada(EstrategiaParticionado):

  # Crea particiones segun el metodo de validacion cruzada.
  # El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
  # Esta funcion devuelve una lista de particiones (clase Particion)
  # TODO: implementar

  def __init__(self, k):
    self.k = k
    super().__init__("Validacion cruzada")

  def creaParticiones(self, datos, seed=None):
    np.random.seed(seed)

    self.numeroParticiones = self.k

    # Generamos una lista con todos los números de datos aleatorios
    indicesAleatorios = np.random.permutation(int(datos.numDatos))

    # Hallamos el tamaño de cada bloque
    tamBloque = int(datos.numDatos / self.k)

    datosSobran = datos.numDatos - (tamBloque * self.k)
    count = 0
    for i in range(self.k):

      train = np.delete(indicesAleatorios, range(i * tamBloque, (i + 1) * tamBloque))
      test = indicesAleatorios[i * tamBloque:(i + 1) * tamBloque]

      # Caso en el que la cuenta es justa
      if datosSobran == 0:
        self.particiones.append(Particion(train, test))

      # Contemplamos el caso de que la division para sacar el numero de subconjuntos no fuese entera
      if datosSobran > 0:
        count += 1
        particionTest = np.append(test, train[(datos.numDatos - tamBloque) - i - 1])
        particionTrain = np.delete(train, (datos.numDatos - tamBloque) - i - 1)
        datosSobran -= 1
        self.particiones.append(Particion(particionTrain, particionTest))

<p>A continuación, mostraremos la ejecución de nuestra estrategia de partcionado validación simple o también llamada K-fold, con los diferentes conjuntos de datos y con diferentes K's. Vamos poder observar en la salida de nuestra celda que todos los valores van a estar una vez en la partición de Test.</p>


In [8]:
print("==============VALIDACIÓN CRUZADA CON LENSES DATA K=5======================")
estrategia1 = ValidacionCruzada(5)
estrategia1.creaParticiones(dataset)
for particion in estrategia1.particiones:
    print(particion)
print("==========================================================================")
print("==============VALIDACIÓN CRUZADA CON TIC-TAC-TOE DATA K=8=================")
estrategia21 = ValidacionCruzada(8)
estrategia21.creaParticiones(dataset2)
for particion in estrategia21.particiones:
    print(particion)
print("=========================================================================")
print("==============VALIDACIÓN CRUZADA CON GERMAN DATA K=4=====================")
estrategia31 = ValidacionCruzada(4)
estrategia31.creaParticiones(dataset3)
for particion in estrategia31.particiones:
    print(particion)
print("=========================================================================")

==============VALIDACIÓN CRUZADA CON LENSES DATA K=5======================
Train: [ 4 18  2 10 12 17  1  3 19 22 15  9 13  7 20 11 14  8 16]
Test:  [21  0  6 23  5]
Train: [21  0  6 23 12 17  1  3 19 22 15  9 13  7 20 11 14  8  5]
Test:  [ 4 18  2 10 16]
Train: [21  0  6 23  4 18  2 10 19 22 15  9 13  7 20 11 14 16  5]
Test:  [12 17  1  3  8]
Train: [21  0  6 23  4 18  2 10 12 17  1  3 13  7 20 11  8 16  5]
Test:  [19 22 15  9 14]
Train: [21  0  6 23  4 18  2 10 12 17  1  3 19 22 15  9 14  8 16  5]
Test:  [13  7 20 11]
==============VALIDACIÓN CRUZADA CON TIC-TAC-TOE DATA K=8=================
Train: [132 488 938 140 910 888 431 770 603 837 118 710 605 931 627 261 730 157
 560 722 242  28 816 460 684 893 491 614 448  59 813 393 894 882  82 185
 511  73 555 745 534 619 142 646  41 581 957  17 558 514 858 337 110 709
 257 926 480 415 473 589 219 724 494 287 592  75 795 826 123 101 384 908
 411 553 723 557  40 155 935 392  27 818  37  12 364 238 228 610 896 530
 639  36 704 674 153 812 517

<h3>Apartado 2: Naive-Bayes</h3>
<p>Es un clasificador de datos que se basa en la regla de Bayes. Donde primero vamos a dividir el conjunto de datos y poder particionarlo en dos subconunto de datos con validación simple o en varios subconjunto de datos con validación cruzada. En el código que mostramos a continuación es lo que van hacer todos los clasificadores que podemos implementar. Los métodos de entrenamiento y clasifica cada clasificador lo hace el clasificador debido a que los métodos que van a utilizar son únicos y los tenemos que implementar en las clases especificas de cada clasificador.</p>
<p>El método <strong>error</strong> va a comprobar los errores que ha obtenido nuestro clasificador, para ver el error que hemos obtenido se hace comparando la última columna de nuestra matriz de datos con la predicción que hemos obtenido en el método de clasifica del clasificador. Si es dintinto la predicción con la última columna de los datos le sumamos uno y lo vamos a dividir entre el número de lineas que tiene el subconjunto de datos Test.</p>
<p>El método <strong>validación</strong> lo que va hacer es realizar los métodos de entrenamiento, clasifica y calcula error del clasificador seguido sin ninguna interrupcion. El método va a comprobar si le estan pasando validación simple o cruzada con el número de partciones que tiene. Si el tamaño es igual a 1 va a llamar a los métodos mencionados anteriormente y va a devolver el error que ha obtenido tras el entrenamiento y la clasificación. Si el tamaño es mayor a 1 hacemos un bucle que cubra todas las partciones para entrenarlas, clasificarlas y obtener su error, despues de la realización de esos métodos vamos a hacer la media aritmetica de todos los errores obtenidos con las diferentes particiones que tenemos en nuestra estrategia.</p>

In [13]:
class Clasificador:
  # Clase abstracta
  __metaclass__ = ABCMeta

  # Metodos abstractos que se implementan en casa clasificador concreto
  @abstractmethod
  # TODO: esta funcion debe ser implementada en cada clasificador concreto
  # datosTrain: matriz numpy con los datos de entrenamiento
  # atributosDiscretos: array bool con la indicatriz de los atributos nominales
  # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion de variables discretas
  def entrenamiento(self, datos, datosTrain, atributosDiscretos, diccionario):
    pass

  @abstractmethod
  # TODO: esta funcion debe ser implementada en cada clasificador concreto
  # devuelve un numpy array con las predicciones
  def clasifica(self, datosTest, atributosDiscretos, diccionario):
    pass

  # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
  # TODO: implementar
  def error(self, datos, pred):
    # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error
    i = 0
    real = datos[:, -1]
    error = 0
    for i in range(len(real)):
      if real[i] != pred[i]:
        error += 1
    err = (error) / (len(real) + 0.0)
    return err

  # Realiza una clasificacion utilizando una estrategia de particionado determinada
  # TODO: implementar esta funcion
  def validacion(self, particionado: object, dataset: object, clasificador: object, seed: object = None) -> object:

    # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
    # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
    # y obtenemos el error en la particion de test i
    # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
    # y obtenemos el error en la particion test. Otra opci�n es repetir la validaci�n simple un n�mero especificado de veces, obteniendo en cada una un error. Finalmente se calcular�a la media.
    errores = 0
    # particionado.creaParticiones(dataset, seed)
    # Comprobamos si es por validación cruzada o simple, por la longitud de la lista de particiones

    particionado.creaParticiones(dataset)

    # Validación Simple
    if len(particionado.particiones) == 1:
      clasificador.entrenamiento(dataset, particionado.particiones[0].indicesTrain)
      pred = clasificador.clasifica(dataset, particionado.particiones[0].indicesTest)
      ret = self.error(dataset.extraeDatos(particionado.particiones[0].indicesTest), pred)
      if ret > 0:
        return ret
      else:
        return 0

    # Validación Cruzada
    else:
      for particion in particionado.particiones:
        clasificador.entrenamiento(dataset, particion.indicesTrain)
        pred = clasificador.clasifica(dataset, particion.indicesTest)
        ret = self.error(dataset.extraeDatos(particion.indicesTest), pred)
        errores += ret
      error = errores / len(particionado.particiones)

      # Devolucion de la media de los errores
      return error

  def matrizConfusion(self, dataset, datosTest, prediccion):

    # Calculamos la matriz de confusion utlizando sk-learn. Solo se calcula en el caso de que la clasificacion sea binaria.
    testData = dataset.extraeDatos(datosTest)
    clase_real = testData[:, -1]

    matriz = confusion_matrix(prediccion, clase_real)

    # La funcion ravel() devuelve todas las estadisticas relacionadas con la matriz de confusion
    tn, fp, fn, tp = matriz.ravel()


    # Calculamos las tasas extraídas de la matriz de confusión
    tpr = tp / (tp + fn)
    fpr = fp / (fp + fn)

    self.lista_tpr.append(tpr)
    self.lista_fpr.append(fpr)

    return matriz

  def curvaROC(self):

    x = np.linspace(0, 1, 100)
    plt.plot(x, x, c='blue')
    for i in range(len(self.lista_fpr)):
        plt.plot(self.lista_fpr[i],self.lista_tpr[i],'ro')
    plt.show()


<h3>Implementación del Clasificador Naive-Bayes</h3>
<p>En la siguiente celda vamos a poder observar el codigo de entrenamiento y clasifica del clasificador de Naive-Bayes, a continuación explicaremos brevemente el funcionamiento de cada uno de los métodos especificos del clasificador. En este claisificador podemos aplicar la regla de Laplace, que es si obtenemos un cero en algunas de las mátrices de conteos de los datos, es decir, a la hora de calcular P(D|H), tendremos que sumar 1 a todas las celdas de conteos de esa P(D|H).</p>
<p>El método <strong>entrenamiento</strong>, lo primero que hace este método es obtener las probabilidades a priori de las clases que hay en el subconjunto de Test. Al calcular esas probabbilades las introducimos en un diccionario para poder utilizarlas más tarde. Ahora vamos a calcular la P(D|H)que se va a calcular de diferentes maneras para datos continuos o discretos
    <ol>
        <li><strong>Atributos discretos</strong>: se calcula haciendo los conteos de las veces que sale la P(D|H) en el subconjunto de datos Train.</li>
        <li><strong>Atributos continuos</strong>: se calcula la media y desviación tipica del subconjunto de datos Train.</li>
    </ol>
Todo esto se mete en una matriz que tiene los diferentes datos y las diferentes clases del conjunto de datos, donde esa matriz la vamos a utilizar para calcular las probabilidades a posteriori con todos las datos que tiene las matrices que hemos creado.</p>
<p>El método <strong> clasifica</strong> dependiendo de los datos que vamos obteniendo del subconjunto de datos de Test, y vamos a obtener las diferentes probabilidades de las distintas clases que tenemos el problema si nos llega ese dato. Esas probabilidades las guardamos en una lista para luego multiplicarlas por los a priori y poder coger la clase que de más probabilidad para guardarla en la lista de las predicciones de nuestro clasificador Naive-Bayes y así despues obtener el error que hemos obtenenido. Este método tambien clasifica de manera distinta los atributos discretos y continuos:
    <ol>
        <li><strong>Atributos discretos</strong>: se calcula obteniendo el número que hay en la matriz de probabilidades a posteriori que hemos creado anteriormente.</li>
        <li><strong>Atributos continuos</strong>: se calcula haciendo la ecuación de la distribución normal.
   </ol>
 </p>

In [14]:
class ClasificadorNaiveBayes(Clasificador):

  def __init__(self, laplace):
    self.laplace = laplace
    self.lista_fpr = []
    self.lista_tpr = []

  def entrenamiento(self, dataset, datosTrain):

    # Cargamos todos los datos de la clase del dataset desde la matriz de datos
    clasesTrain = dataset.extraeDatos(datosTrain)
    self.numClases = clasesTrain[:, -1]

    # Contamos las apariciones de cada uno para luego calcular la probabilidad a priori de cada clase
    counter = Counter(self.numClases)

    # Calculamos la probabilidad de la clase y lo metemos en un diccionario ordenado segun el numero
    # correspondiente a cada clase asignado en el diccionario
    self.dictPrioris = {}
    for k in counter:
      k = int(k)
      counter[k] = counter[k] / len(self.numClases)
      self.dictPrioris[k] = counter[k]

    # Aqui ordenamos el diccionario para que esten en el mismo orden de como extraemos los datos del dataset
    self.dictPrioris = SortedDict(self.dictPrioris)

    # Calcular tablas de probabilidades del entrenamiento. Tenemos que calcular por cada atributo una cuenta
    # de las apariciones en cada clase
    # Creamos una lista de matrices, donde vamos almacenar todos los datos que hemos obtenido en los datos de Test
    self.posteriori = np.zeros(len(dataset.nombreAtributos) - 1, dtype=object)

    # Recorremos todos los datos de la matriz sin llegar a la clase
    for i in range(len(dataset.nombreAtributos) - 1):

      # Si el dato que obtenemos es Nominal haremos el recuento de todas las veces que sale la P(D|H)
      if dataset.nominalAtributos[i] == True:

        # Creamos una matriz de tamaño X: Número de Atributos menos la clase Y: Número de clases
        post = np.zeros((len(dataset.listaDicts[i]), len(dataset.listaDicts[-1])))

        # Aqui contamos todos las datos que queremos del datos Train para construir la matriz de entrenamiento
        for c in range(len(dataset.listaDicts[-1])):
          datosEnt = dataset.extraeDatos(datosTrain)
          dat = datosEnt[:, i]
          repes = Counter(dat[datosEnt[:, -1] == c])
          for r in repes:
            post[int(r), c] = repes[r]
          if self.laplace == True:
            self.posteriori[i] = post + 1
          else:
            self.posteriori[i] = post

      # Si el dato es Continuo obtendremos la media y la desviación tipica de la clase
      else:

        # Creamos una matriz de X: Los datos de Media y Desivación típica Y: Número de clases
        post = np.zeros((2, len(dataset.listaDicts[-1])))

        # Aqui obtenemos la media y desviación tipica de cada clase, despues de tener los datos de entrenamiento
        for c in range(len(dataset.listaDicts[-1])):
          datosEnt = dataset.extraeDatos(datosTrain)
          dat = datosEnt[:, i]
          datos = dat[datosEnt[:, -1] == c]
          post[0][c] = np.mean(datos)
          post[1][c] = np.std(datos)
        self.posteriori[i] = post


    # Calculamos los valores de los posteriori de todos las tablas anteriores
    for i in range(len(dataset.listaDicts) - 1):
      if dataset.nominalAtributos[i] == True:
        self.posteriori[i] /= sum(self.posteriori[i])




  def clasifica(self, dataset, datosTest):
    acum_probs = 1
    self.prediccion = []
    datTest = dataset.extraeDatos(datosTest)

    # Ahora vamos a estudiar la probabilidad de la clase con los datos obtenidos en el entrenamiento
    # Recorremos todos las datos de la matriz de los datos Test
    for dato in datTest:
      mapa = []
      # Aqui obtenemos los prioris de cada clase para poder obtener la probabilidad de cada una
      for clase in range(len(self.dictPrioris)):
        listaVerosimilitudes = []
        # Aqui obtenemos cada valor posteriori de nuestro entrenamiento de los datos, es decir, P(D|H)
        for atributo in range(len(self.posteriori)):
          if dataset.nominalAtributos[atributo] == True:
            prob = self.posteriori[atributo][int(dato[atributo])][clase]
            listaVerosimilitudes.append(prob)

          # Aqui obtenemos la probabilidad de los atibutos continuos
          else:
            # Hacemos la formula de la distribucion normal
            exp1 = 1 / (self.posteriori[atributo][1][clase] * math.sqrt(2 * math.pi))
            exp2 = np.power((dato[atributo] - self.posteriori[atributo][0][clase]), 2)
            exp3 = np.power(self.posteriori[atributo][1][clase], 2)
            exp4 = exp2 / exp3
            exp4 = math.exp((-1 / 2) * exp4)
            prob = exp1 * exp4
            listaVerosimilitudes.append(prob)

        for verosimilitud in listaVerosimilitudes:
          acum_probs *= verosimilitud
        acum_probs *= self.dictPrioris.get(clase)
        mapa.append(acum_probs)
        acum_probs = 1

      # Aqui obtenemos la predicción de mayor probabilidad y la guardamos en nuestra lista de predicciones
      self.prediccion.append(np.argmax(mapa))


    # Devolvemos la lista con la predicción de nuestro clasifica
    return self.prediccion

<p>A continuación, vamos a mostrar una ejecución del claisificador de Naive-Bayes con las diferentes validaciones y los diferentes conjuntos de datos que tenemos, al final de esto mostraremos la probabilidad que hemos obtenido</p>

In [15]:
print("==============NAIVE-BAYES CON LENSES DATA ======================")
print("==============CON LAPLACE Y VALIDACION SIMPLE===================")
nb = ClasificadorNaiveBayes(True)
error = nb.validacion(estrategia,dataset,nb)
print("ERROR OBTENIDO:",error)
print("==============CON LAPLACE Y VALIDACION CRUZADA==================")
nb = ClasificadorNaiveBayes(True)
error = nb.validacion(estrategia1,dataset,nb)
print("ERROR OBTENIDO:",error)
print("==============SIN LAPLACE Y VALIDACION SIMPLE===================")
nb = ClasificadorNaiveBayes(False)
error = nb.validacion(estrategia,dataset,nb)
print("ERROR OBTENIDO:",error)
print("==============SIN LAPLACE Y VALIDACION CRUZADA==================")
nb = ClasificadorNaiveBayes(False)
error = nb.validacion(estrategia1,dataset,nb)
print("ERROR OBTENIDO:",error)
print("================================================================")
print("==============NAIVE-BAYES CON TIC-TAC-TOE DATA==================")
print("==============CON LAPLACE Y VALIDACION SIMPLE===================")
nb = ClasificadorNaiveBayes(True)
error = nb.validacion(estrategia2,dataset2,nb)
print("ERROR OBTENIDO:",error)
print("==============CON LAPLACE Y VALIDACION CRUZADA==================")
nb = ClasificadorNaiveBayes(True)
error = nb.validacion(estrategia21,dataset2,nb)
print("ERROR OBTENIDO:",error)
print("==============SIN LAPLACE Y VALIDACION SIMPLE===================")
nb = ClasificadorNaiveBayes(False)
error = nb.validacion(estrategia2,dataset2,nb)
print("ERROR OBTENIDO:",error)
print("==============SIN LAPLACE Y VALIDACION CRUZADA==================")
nb = ClasificadorNaiveBayes(False)
error = nb.validacion(estrategia21,dataset2,nb)
print("ERROR OBTENIDO:",error)
print("================================================================")
print("==============NAIVE-BAYES CON GERMAN DATA ======================")
print("==============CON LAPLACE Y VALIDACION SIMPLE===================")
nb = ClasificadorNaiveBayes(True)
error = nb.validacion(estrategia3,dataset3,nb)
print("ERROR OBTENIDO:",error)
print("==============CON LAPLACE Y VALIDACION CRUZADA==================")
nb = ClasificadorNaiveBayes(True)
error = nb.validacion(estrategia31,dataset3,nb)
print("ERROR OBTENIDO:",error)
print("==============SIN LAPLACE Y VALIDACION SIMPLE===================")
nb = ClasificadorNaiveBayes(False)
error = nb.validacion(estrategia3,dataset3,nb)
print("ERROR OBTENIDO:",error)
print("==============SIN LAPLACE Y VALIDACION CRUZADA==================")
nb = ClasificadorNaiveBayes(False)
error = nb.validacion(estrategia31,dataset3,nb)
print("ERROR OBTENIDO:",error)
print("================================================================")

==============NAIVE-BAYES CON LENSES DATA ======================
==============CON LAPLACE Y VALIDACION SIMPLE===================
ERROR OBTENIDO: 0.375
==============CON LAPLACE Y VALIDACION CRUZADA==================
ERROR OBTENIDO: 0.3375000000000001
==============SIN LAPLACE Y VALIDACION SIMPLE===================
ERROR OBTENIDO: 0.25
==============SIN LAPLACE Y VALIDACION CRUZADA==================
ERROR OBTENIDO: 0.3360000000000001
==============NAIVE-BAYES CON TIC-TAC-TOE DATA==================
==============CON LAPLACE Y VALIDACION SIMPLE===================
ERROR OBTENIDO: 0.2916666666666667
==============CON LAPLACE Y VALIDACION CRUZADA==================
ERROR OBTENIDO: 0.3019498424369748
==============SIN LAPLACE Y VALIDACION SIMPLE===================
ERROR OBTENIDO: 0.2916666666666667
==============SIN LAPLACE Y VALIDACION CRUZADA==================
ERROR OBTENIDO: 0.3025280112044818
==============NAIVE-BAYES CON GERMAN DATA ======================
==============CON LAPLACE Y VALI

<h3>Apartado 3: Scikit-Learn</h3>
<p>Es una libreria de python que nos proporciona diferentes implementaciones del clasificador de Naive-Bayes. A continuación, vamos a explicar lo que hace cada método que hemos implementado en la celda de abajo con la libreria de Scikit-Learn.</p>
<p>El método <strong>validacion_simple_sklearn</strong> donde le introducimos por parametro el conjunto de datos del cual queremos hacer la validación y el porcentaje que queremos tener del subconjunto de entrenamiento. La función de sklearn nos va a devolver los subconjuntos de datos de entrenamiento y de clasificación.</p>
<p>El método <strong>validacion_cruzada_sklearn</strong> donde le introducimos por parametro el conjunto de datos del cual queremos hacer la validación y el número de partciones que vamos a hacer del conjunto de datos. El método nos va a devolver la lista de particiones que hemos obtenido con el método de sklearn.</p>

In [16]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from EstrategiaParticionado import Particion
def validacion_simple_sklearn(dataset, porcentaje):

    # Matriz con los atributos
    X = dataset.datos[:, :-1]

    # Array con las clases
    y = dataset.datos[:, -1]

    # Realizamos la divison en train-test, X_train es la partición sobre la que se va a entrenar e X_test sobre la que se va a clasificar
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=porcentaje, test_size=1 - porcentaje, shuffle=True)

    return X_train, X_test, y_train, y_test

def validacion_cruzada_sklearn(dataset, k):

    # Matriz con los atributos
    X = dataset.datos[:, :-1]

    # Array con las clases
    y = dataset.datos[:, -1]

    kf = KFold(n_splits=k, shuffle=True)

    particiones = []

    for train_index, test_index in kf.split(X,y):
        particiones.append(Particion(train_index,test_index))

    return particiones

In [17]:
print("==============SKLEARN VALIDACIÓN SIMPLE 70% LENSES DATA==================")
x_train, x_test, y_train, y_test = validacion_simple_sklearn(dataset, 0.7)
print("TRAIN:\n", x_train)
print("TEST:\n", x_test)
print("==============SKLEARN VALIDACIÓN CRUZADA K=5 LENSES DATA=================")
particiones = validacion_cruzada_sklearn(dataset,5)
for particion in particiones:
    print(particion)
print("=========================================================================")
print("==============SKLEARN VALIDACIÓN SIMPLE 80% TIC-TAC-TOE DATA=============")
x_train1, x_test1, y_train1, y_test1 = validacion_simple_sklearn(dataset2, 0.8)
print("TRAIN:\n", x_train1)
print("TEST:\n", x_test1)
print("==============SKLEARN VALIDACIÓN CRUZADA K=8 TIC-TAC-TOE DATA============")
particiones = validacion_cruzada_sklearn(dataset2,8)
for particion in particiones:
    print(particion)
print("=========================================================================")
print("==============SKLEARN VALIDACIÓN SIMPLE 75% GERMAN DATA==================")
x_train2, x_test2, y_train2, y_test2 = validacion_simple_sklearn(dataset3, 0.75)
print("TRAIN:\n", x_train2)
print("TEST:\n", x_test2)
print("==============SKLEARN VALIDACIÓN CRUZADA K=4 GERMAN DATA=================")
particiones = validacion_cruzada_sklearn(dataset3,4)
for particion in particiones:
    print(particion)
print("=========================================================================")

==============SKLEARN VALIDACIÓN SIMPLE 70% LENSES DATA==================
TRAIN:
 [[1. 0. 1. 1.]
 [1. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 1. 1. 1.]
 [0. 1. 1. 1.]
 [0. 0. 1. 0.]
 [2. 1. 1. 0.]
 [2. 0. 0. 0.]
 [0. 0. 1. 1.]
 [2. 1. 0. 1.]
 [2. 0. 0. 1.]
 [0. 1. 1. 0.]
 [0. 1. 0. 1.]
 [2. 1. 0. 0.]]
TEST:
 [[1. 1. 0. 1.]
 [2. 1. 1. 1.]
 [2. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 1.]
 [2. 0. 1. 1.]
 [1. 1. 1. 0.]
 [0. 0. 0. 0.]]
==============SKLEARN VALIDACIÓN CRUZADA K=5 LENSES DATA=================
Train: [ 0  2  3  4  5  6  7  8  9 10 12 13 14 16 17 18 19 20 23]
Test:  [ 1 11 15 21 22]
Train: [ 0  1  2  3  4  6  7  8 10 11 12 15 16 18 19 20 21 22 23]
Test:  [ 5  9 13 14 17]
Train: [ 0  1  4  5  6  8  9 10 11 12 13 14 15 17 18 19 20 21 22]
Test:  [ 2  3  7 16 23]
Train: [ 0  1  2  3  4  5  7  9 11 13 14 15 16 17 19 20 21 22 23]
Test:  [ 6  8 10 12 18]
Train: [ 1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 21 22 23]
Test:  [ 0  4 19 20]
==============SKLEARN VAL

<h3>Clasificador Naive-Bayes con Sklearn</h3>
<p>Ahora vamos a realizar pruebas con la implementación que nos da sklearn del clasificador de Naive-Bayes, anteriormente hemos realizado las diferentes estrategias de particionado, por lo tanto, ahora solo nos falta introducir esos subconjuntos a los métodos que hemos creado para tener la implementacion de sklearn y que la libreria se encargue de hacer el entrenamiento y la clasificación del conjunto de datos. A continuación, explicaremos brevemente cada uno de los métodos que hemos creado.</p>
<p>El método <strong>nb_sklearn</strong>: en este método introducimos como parametro el subconjunto de datos de entrenamiento y de clasificacion, el tipo que queremos calcular y si se va a utilizar la regla de Laplace, donde el tipo va estar predefinido a Multinominal y laplace va estar definido a True. Este método va a devolver la predicción de las clases.</p>
<p>El método <strong>error</strong>: este método va a calcular los errores que vamos a obtener el pporcentaje de error que hemos obtenido con el clasificador Naive-Bayes.</p>

In [19]:
def validacion_simple_sklearn(dataset, porcentaje):

    # Matriz con los atributos
    X = dataset.datos[:, :-1]

    # Array con las clases
    y = dataset.datos[:, -1]

    # Realizamos la divison en train-test, X_train es la partición sobre la que se va a entrenar e X_test sobre la que se va a clasificar
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=porcentaje, test_size=1 - porcentaje, shuffle=True)

    return X_train, X_test, y_train, y_test

def validacion_cruzada_sklearn(dataset, k):

    # Matriz con los atributos
    X = dataset.datos[:, :-1]

    # Array con las clases
    y = dataset.datos[:, -1]

    kf = KFold(n_splits=k, shuffle=True)

    particiones = []

    for train_index, test_index in kf.split(X,y):
        particiones.append(Particion(train_index,test_index))

    return particiones

def nb_sklearn(x_train, y_train, x_test, tipo="Multinomial", laplace=True):

    if tipo == "Gaussian":
        if laplace == True:
            clf = GaussianNB(alpha=1.0)
        else:
            clf = GaussianNB()

    elif tipo == "Multinomial":
        if laplace == True:
            clf = MultinomialNB(alpha=1.0, fit_prior = True, class_prior = None)
        else:
            clf = MultinomialNB(fit_prior=True, class_prior=False)
    else:
        print("Error, clasificador no valido. Utilizar GaussianNB o MultinomialNB")
        return

    # Entrenamos el modelo
    clf.fit(x_train, y_train)

    # Clasificacion
    prediccion = clf.predict(x_test)

    return prediccion

def nb_sklearn_validacion_cruzada(x_train, y_train, k):

    clf = MultinomialNB(alpha = 1.0, fit_prior = True, class_prior = None)

    error = cross_val_score(clf, x_train, y_train, cv = k)

    return error



def error(clases_predichas, clases_reales):

    return 1 - np.sum(np.equal(clases_predichas, clases_reales)) / len(clases_predichas)

In [39]:
print("==============SKLEARN NAIVE-BAYES LENSES DATA==================")
print("==============CON LAPLACE Y VALIDACION SIMPLE==================")
x_train, x_test, y_train, y_test = validacion_simple_sklearn(dataset, 0.7)
pred = nb_sklearn(x_train, y_train, x_test)
errornb = error(pred, x_test[:,-1])
print("ERROR OBTENIDO:", errornb)
print("==============SIN LAPLACE Y VALIDACION SIMPLE==================")
x_train, x_test, y_train, y_test = validacion_simple_sklearn(dataset, 0.7)
pred = nb_sklearn(x_train, y_train, x_test, "Multinomial",False)
errornb = error(pred, x_test[:,-1])
print("ERROR OBTENIDO:", errornb)
print("==============SKLEARN NAIVE-BAYES TIC-TAC-TOE DATA=============")
print("==============CON LAPLACE Y VALIDACION SIMPLE==================")
x_train2, x_test2, y_train2, y_test2 = validacion_simple_sklearn(dataset2, 0.8)
pred = nb_sklearn(x_train2, y_train2, x_test2)
errornb2 = error(pred, x_test2[:,-1])
print("ERROR OBTENIDO:", errornb)
print("==============SIN LAPLACE Y VALIDACION SIMPLE==================")
pred = nb_sklearn(x_train2, y_train2, x_test2, "Multinomial",False)
errornb2 = error(pred, x_test2[:,-1])
print("ERROR OBTENIDO:", errornb)
print("==============SKLEARN NAIVE-BAYES GERMAN DATA=============")
print("==============CON LAPLACE Y VALIDACION SIMPLE==================")
x_train3, x_test3, y_train3, y_test3 = validacion_simple_sklearn(dataset3, 0.75)
pred = nb_sklearn(x_train3, y_train3, x_test3)
errornb2 = error(pred, x_test3[:,-1])
print("ERROR OBTENIDO:", errornb)
print("==============SIN LAPLACE Y VALIDACION SIMPLE==================")
pred = nb_sklearn(x_train3, y_train3, x_test3, "Multinomial",False)
errornb2 = error(pred, x_test3[:,-1])
print("ERROR OBTENIDO:", errornb)

==============SKLEARN NAIVE-BAYES LENSES DATA==================
==============CON LAPLACE Y VALIDACION SIMPLE==================
ERROR OBTENIDO: 1.0
==============SIN LAPLACE Y VALIDACION SIMPLE==================


TypeError: object of type 'bool' has no len()

<h3>Apartado 4:Evaluación de hipótesis mediante Análisis ROC</h3>
<p>La curva ROC es una representación gráfica de la sensibilidad a la especifidad de un clasificador, en esta práctica este análisis lo vamos a realizar del clasificador implementado que es Naive-Bayes. En este gráfico se representan los verdaderos postivos frente a los falsos positivos.</p>
<p>Es una herramienta que nos proporciona la selección de modelos más óptimos y descartar los menos óptimos.</p>
<p>A continuación, mostraremos la implementación que hemos realizado en para crear este análisis ROC:</p>
<ol>
    <li>El primer paso es crear la <strong> matriz de confusión</strong>, donde esta matriz la hemos utilizado con un método de la libreria de sklearn que nos dibuja la matriz de confusion para los datos que queremos del conjunto de datos. Despues de haber creado la matriz calculamos los valores de verdaderos positivos, falsos positivos, falsos negativos y verdaderos negativos y, por último, calculamos las tasas de la matriz de confusion y las guardamos en una lista. </li>


<p>Vamos a mostrar a continuación una ejecución del anterior codigo para verlo con diferentes datasets y diferentes predicciones.</p>

TypeError: matrizConfusion() missing 1 required positional argument: 'prediccion'

<ol>
2. En segundo lugar, debemos sacar la gráfica de la curva ROC. Esta gráfica lo sacamos con la libreria pyplot, mas concretamente, con matplotlib. Donde en el eje Y pondremos los valores TPR y en el eje de las X pondremos los valores de FPR.
</ol>

<p> A contnuación, mostraremos todas las curvas ROC para todos los conjuntos de datos que tenemos y asi poder ver si nuestro clasificador es óptimo para este tipo de datos.</p>